In [3]:
import os
import re
import requests
import subprocess
import shutil
import time
import sys
import ipywidgets as widgets
from IPython.display import display, clear_output
from urllib.parse import urlparse, unquote
from google.colab import drive

# --- CONFIGURATION ---
DRIVE_TV_PATH = "TV Shows"
DRIVE_MOVIE_PATH = "Movies"
DRIVE_YOUTUBE_PATH = "YouTube"
MIN_FILE_SIZE_MB = 10
KEEP_EXTENSIONS = {'.srt', '.ass', '.sub', '.vtt'}

# --- GLOBAL STATE ---
report_log = {"TV": [], "Movies": [], "YouTube": [], "Failed": []}
start_time_global = 0

# --- UI ELEMENTS ---
# We define these globally so they persist
token_gf = widgets.Text(description='Gofile:', placeholder='Optional')
token_rd = widgets.Text(description='RD Token:', placeholder='Real-Debrid API Key')
# NEW: Show Name Override Widget
show_name_override = widgets.Text(description='Show Name Override:', placeholder='Optional (Forces Show Name)', style={'description_width': 'initial'})

text_area = widgets.Textarea(description='Links:', placeholder='Paste Links Here...', layout=widgets.Layout(width='98%', height='150px'))
btn = widgets.Button(description="Start Download", button_style='success', icon='download')
btn_subs = widgets.Button(description="Download Subtitles Only", button_style='info', icon='closed-captioning')
progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=100.0, description='Idle', bar_style='info', layout=widgets.Layout(width='98%'))

# Container for all inputs
input_ui = widgets.VBox([
    widgets.HTML("<h3>🚀 Ultimate Downloader v3.6 (Manual Override)</h3>"),
    widgets.HBox([token_gf, token_rd]),
    show_name_override, # Added to UI
    text_area,
    widgets.HBox([btn, btn_subs]),
    progress_bar,
    widgets.HTML("<hr>")
])

# --- HELPER: TEXT SANITISATION ---
def sanitize_filename(name):
    name = unquote(name)
    name = re.sub(r'[<>:"/\\|?*]', '_', name)
    name = "".join(c for c in name if c.isprintable())
    name = re.sub(r'[\s_]+', ' ', name).strip()
    return name

def clean_show_name(name):
    name = re.sub(r'(?i)(?:\[?\s*(?:ENG\s*SUB|ENGSUB|FULL)\s*\]?)', '', name)
    name = re.sub(r'[\[\]\(\)《》【】]', ' ', name)
    name = re.sub(r'[|._-]', ' ', name)
    name = re.sub(r'(?i)\s+\b(END|FINALE|FINAL)\b$', '', name)
    clean = name.strip()
    return clean if clean else "Unknown Show"

def determine_destination_path(filename, source="generic"):
    filename = sanitize_filename(filename)

    # Check Override First
    manual_show_name = show_name_override.value.strip()

    sxe_strict = re.search(r'(?i)\bS(\d{1,2})E(\d{1,2})\b', filename)
    sxe_loose = re.search(r'(?i)\b(?:Ep?|Episode)[ ._]?(\d{1,3})\b', filename)
    sxe_asian = re.search(r'第(\d+)集', filename)

    season_num = 1
    episode_num = 1
    category = "Movies" # Default
    is_tv = False

    if sxe_strict:
        season_num = int(sxe_strict.group(1))
        episode_num = int(sxe_strict.group(2))
        show_name = clean_show_name(filename[:sxe_strict.start()])
        is_tv = True
    elif sxe_loose:
        episode_num = int(sxe_loose.group(1))
        raw_before = filename[:sxe_loose.start()]
        show_name = clean_show_name(raw_before)
        if len(show_name) < 2:
            raw_after = os.path.splitext(filename[sxe_loose.end():])[0]
            show_name = clean_show_name(raw_after)
        is_tv = True
    elif sxe_asian:
        episode_num = int(sxe_asian.group(1))
        show_name = clean_show_name(filename[:sxe_asian.start()])
        if len(show_name) < 2:
            show_name = clean_show_name(os.path.splitext(filename[sxe_asian.end():])[0])
        is_tv = True

    # FORCE OVERRIDE logic
    if manual_show_name:
        show_name = manual_show_name
        is_tv = True # If user provides a show name, assume it's TV unless manually categorized otherwise logic needed (but usually safe assumption here)
        category = "TV"
    elif is_tv:
        category = "TV"
    else:
        # Movie Logic (Only if no override and no TV pattern found)
        year_match = re.search(r'\b(19|20)\d{2}\b', filename)
        if year_match:
            movie_name = clean_show_name(filename[:year_match.start()])
            base_path = f"/content/drive/My Drive/{DRIVE_MOVIE_PATH}"
            full_dir = os.path.join(base_path, movie_name)
        elif source == "youtube":
            base_path = f"/content/drive/My Drive/{DRIVE_YOUTUBE_PATH}"
            full_dir = base_path
            category = "YouTube"
            return os.path.join(full_dir, filename), category
        else:
            movie_name = clean_show_name(os.path.splitext(filename)[0])
            base_path = f"/content/drive/My Drive/{DRIVE_MOVIE_PATH}"
            full_dir = os.path.join(base_path, movie_name)

        if not os.path.exists(full_dir): os.makedirs(full_dir, exist_ok=True)
        return os.path.join(full_dir, filename), category

    # TV PATH BUILDER (With Override Applied)
    base_path = f"/content/drive/My Drive/{DRIVE_TV_PATH}"
    season_folder = f"Season {season_num:02d}"
    full_dir = os.path.join(base_path, show_name, season_folder)

    _, ext = os.path.splitext(filename)
    new_filename = f"{show_name} - S{season_num:02d}E{episode_num:02d}{ext}"

    if not os.path.exists(full_dir): os.makedirs(full_dir, exist_ok=True)
    return os.path.join(full_dir, new_filename), category

# --- SETUP ---
def setup_environment():
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')
    for p in [DRIVE_TV_PATH, DRIVE_MOVIE_PATH, DRIVE_YOUTUBE_PATH]:
        full_p = f"/content/drive/My Drive/{p}"
        if not os.path.exists(full_p): os.makedirs(full_p)
    try: import yt_dlp
    except ImportError:
        print("🛠️ Installing yt-dlp...")
        subprocess.run("pip install yt-dlp", shell=True, check=True, stdout=subprocess.DEVNULL)
    if not shutil.which('aria2c'):
        print("🛠️ Installing tools...")
        subprocess.run("apt-get update -qq", shell=True)
        subprocess.run("apt-get install -y aria2 unrar p7zip-full ffmpeg", shell=True, check=True, stdout=subprocess.DEVNULL)

# --- DOWNLOADERS ---
def ytdl_hook(d):
    if d['status'] == 'downloading':
        try:
            p = d.get('_percent_str', '0%').replace('%','')
            progress_bar.value = float(p)
            progress_bar.description = f"YT: {p}%"
        except: pass
    elif d['status'] == 'finished':
        progress_bar.value = 100
        progress_bar.description = "Done!"

def process_youtube_link(url, mode="video"):
    import yt_dlp
    print(f"   ▶️ Processing YouTube: {url}")
    progress_bar.value = 0
    progress_bar.description = "Starting..."

    ydl_opts = {
        'outtmpl': '/content/%(title)s.%(ext)s',
        'quiet': True, 'no_warnings': True, 'restrictfilenames': True, 'ignoreerrors': True,
        'writesubtitles': True, 'writeautomaticsub': False,
        'subtitleslangs': ['en.*', 'vi'], 'subtitlesformat': 'srt',
        'progress_hooks': [ytdl_hook],
        'noprogress': True
    }
    if mode == "video":
        ydl_opts['format'] = 'bestvideo+bestaudio/best'
        ydl_opts['merge_output_format'] = 'mkv'
    else: ydl_opts['skip_download'] = True

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            entries = list(info['entries']) if 'entries' in info else [info]
            if 'entries' in info: print(f"   📜 Playlist: {info.get('title', 'Unknown')} ({len(entries)} items)")

            for i, entry in enumerate(entries, 1):
                if not entry: continue
                print(f"      [{i}/{len(entries)}] ⬇️ Downloading: {entry.get('title', 'Unknown')}")
                try:
                    before_files = set(os.listdir('/content/'))
                    ydl.download([entry.get('webpage_url', entry.get('url'))])
                    after_files = set(os.listdir('/content/'))
                    new_files = list(after_files - before_files)

                    if not new_files: print("      ⚠️ No new files found."); continue
                    for f in new_files:
                        if f.endswith(('.part', '.ytdl')): continue
                        full_path = os.path.join('/content/', f)
                        if mode == "subs_only" and not f.endswith('.srt'):
                            if os.path.exists(full_path): os.remove(full_path)
                            continue
                        handle_file_processing(full_path, source="youtube")
                except Exception as e: print(f"      ❌ Error: {e}")
    except Exception as e:
        print(f"   ❌ YouTube Failed: {e}")
        report_log["Failed"].append(url)
    progress_bar.description = "Idle"

def download_with_aria2(url, filename, dest_folder, cookie=None):
    filename = sanitize_filename(filename)
    final_path = os.path.join(dest_folder, filename)
    if os.path.exists(final_path) and os.path.getsize(final_path) > 1024*1024: return final_path

    print(f"   ⬇️ Downloading: {filename}")
    progress_bar.description = "Aria2 DL..."
    progress_bar.bar_style = 'warning'

    cmd = ['aria2c', url, '-d', dest_folder, '-o', filename, '-x', '16', '-s', '16', '-k', '1M', '-c', '--file-allocation=none', '--user-agent', 'Mozilla/5.0']
    if cookie: cmd.extend(['--header', f'Cookie: accountToken={cookie}'])

    for attempt in range(1, 4):
        res = subprocess.run(cmd, capture_output=True, text=True)
        if res.returncode == 0 and os.path.exists(final_path):
            progress_bar.bar_style = 'info'
            return final_path
        else:
            if attempt < 3: time.sleep(2**attempt)
            else: print(f"      ❌ Aria2 Error: {res.stderr.strip()}")

    progress_bar.bar_style = 'info'
    return None

def handle_file_processing(file_path, source="generic"):
    if not file_path or not os.path.exists(file_path): return
    filename = os.path.basename(file_path)
    _, ext = os.path.splitext(filename)

    if ext not in ['.rar', '.zip', '.7z']:
        processing_name = filename
        if ext == '.srt':
            parts = filename.split('.')
            if len(parts) >= 3 and len(parts[-2]) in [2, 3]: processing_name = ".".join(parts[:-2]) + ext

        final_dest, cat = determine_destination_path(processing_name, source)
        if ext == '.srt':
            parts = filename.split('.')
            lang = parts[-2] if len(parts) >= 3 and len(parts[-2]) in [2, 3] else ""
            base = os.path.splitext(final_dest)[0]
            final_dest = f"{base}.{lang}.srt" if lang else f"{base}.srt"

        if os.path.exists(final_dest): os.remove(final_dest)
        shutil.move(file_path, final_dest)
        print(f"   ✨ Moved to {cat}: {os.path.basename(final_dest)}")
        report_log[cat].append(os.path.basename(final_dest))
        return

    print(f"   📦 Extracting: {filename}")
    progress_bar.description = "Extracting..."
    extract_temp = "/content/temp_extract"
    if os.path.exists(extract_temp): shutil.rmtree(extract_temp)
    os.makedirs(extract_temp)

    try:
        cmd = ['unrar', 'x', '-o+', file_path, extract_temp] if '.rar' in ext else ['7z', 'x', '-y', file_path, f'-o{extract_temp}']
        subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        for root, dirs, files in os.walk(extract_temp):
            for f in files:
                full_path = os.path.join(root, f)
                if os.path.getsize(full_path) < MIN_FILE_SIZE_MB * 1024 * 1024 and not f.endswith(tuple(KEEP_EXTENSIONS)): continue

                final_dest, cat = determine_destination_path(f, source)
                if os.path.exists(final_dest): os.remove(final_dest)
                shutil.move(full_path, final_dest)
                print(f"      -> Extracted: {os.path.basename(final_dest)}")
                report_log[cat].append(os.path.basename(final_dest))
    except Exception as e: print(f"   ❌ Extraction Error: {e}")
    finally:
        if os.path.exists(extract_temp): shutil.rmtree(extract_temp)
        os.remove(file_path)
        progress_bar.description = "Idle"

# --- RESOLVERS & EXECUTION ---
def get_gofile_session(token):
    s = requests.Session()
    s.headers.update({'User-Agent': 'Mozilla/5.0'})
    t = {'token': token, 'wt': "4fd6sg89d7s6"}
    if not token:
        try: t['token'] = s.post("https://api.gofile.io/accounts", json={}).json()['data']['token']
        except: pass
    return s, t

def resolve_gofile(url, s, t):
    try:
        cid = re.search(r'gofile\.io/d/([a-zA-Z0-9]+)', url).group(1)
        r = s.get(f"https://api.gofile.io/contents/{cid}", params={'wt': t['wt']}, headers={'Authorization': f"Bearer {t['token']}"}).json()
        if r['status'] == 'ok': return [(c['link'], c['name']) for c in r['data']['children'].values() if c.get('link')]
    except: pass
    return []

def resolve_pixeldrain(url, s):
    try:
        fid = re.search(r'pixeldrain\.com/u/([a-zA-Z0-9]+)', url).group(1)
        name = s.get(f"https://pixeldrain.com/api/file/{fid}/info").json().get('name', f"pixeldrain_{fid}")
        return [(f"https://pixeldrain.com/api/file/{fid}?download", sanitize_filename(name))]
    except: return []

def process_rd_link(link, key):
    h = {"Authorization": f"Bearer {key}"}
    if "magnet:?" in link:
        print("   🧲 Resolving Magnet...")
        try:
            r = requests.post("https://api.real-debrid.com/rest/1.0/torrents/addMagnet", data={"magnet": link}, headers=h).json()
            requests.post(f"https://api.real-debrid.com/rest/1.0/torrents/selectFiles/{r['id']}", data={"files": "all"}, headers=h)
            for _ in range(30): # Wait max 60s
                i = requests.get(f"https://api.real-debrid.com/rest/1.0/torrents/info/{r['id']}", headers=h).json()
                if i['status'] == 'downloaded':
                    for l in i['links']: process_rd_link(l, key)
                    return
                time.sleep(2)
            print("   ❌ Magnet not cached.")
        except: print("   ❌ Magnet Error")
        return
    try:
        d = requests.post("https://api.real-debrid.com/rest/1.0/unrestrict/link", data={"link": link}, headers=h).json()
        f = download_with_aria2(d['download'], d['filename'], "/content/")
        handle_file_processing(f)
    except: print("   ❌ RD Link Error")

def execute_batch(mode):
    global start_time_global
    start_time_global = time.time()

    # Clear logs but keeping input static at top
    clear_output(wait=True)
    display(input_ui)

    btn.disabled = True
    btn_subs.disabled = True
    print(f"\n🚀 Initializing... (Mode: {mode})")

    try:
        setup_environment()
        for k in report_log: report_log[k] = []

        s, t = get_gofile_session(token_gf.value.strip())
        rd = token_rd.value.strip()
        urls = [x.strip() for x in text_area.value.split('\n') if x.strip()]

        print(f"🚀 Processing {len(urls)} links...\n")

        for i, url in enumerate(urls, 1):
            print(f"--- Link [{i}/{len(urls)}] ---")
            if "youtube.com" in url or "youtu.be" in url: process_youtube_link(url, mode)
            elif "gofile.io" in url:
                for u, n in resolve_gofile(url, s, t):
                    f = download_with_aria2(u, n, "/content/", t.get('token'))
                    handle_file_processing(f)
            elif "pixeldrain.com" in url:
                for u, n in resolve_pixeldrain(url, s):
                    f = download_with_aria2(u, n, "/content/")
                    handle_file_processing(f)
            elif "magnet:?" in url or (rd and "http" in url):
                if rd: process_rd_link(url, rd)
                else: print("   ❌ RD Token Required")
            else:
                f = download_with_aria2(url, os.path.basename(unquote(urlparse(url).path)), "/content/")
                handle_file_processing(f)

        elapsed = time.time() - start_time_global
        print(f"\n✅ All Tasks Finished ({int(elapsed//60)}m {int(elapsed%60)}s)")

    except Exception as e: print(f"\n❌ Critical Error: {e}")
    finally:
        btn.disabled = False
        btn_subs.disabled = False

# --- BINDINGS ---
btn.on_click(lambda b: execute_batch("video"))
btn_subs.on_click(lambda b: execute_batch("subs_only"))
display(input_ui)


🚀 Initializing... (Mode: video)
🚀 Processing 1 links...

--- Link [1/1] ---
   ▶️ Processing YouTube: https://www.youtube.com/watch?v=9XbwYHh_7hE
      [1/1] ⬇️ Downloading: 【同一屋檐下 第一季】EP01 上篇 | 合宿首日刷碗事件笑喷邓紫棋 | 邓紫棋/陈建斌/魏大勋/李诞/曾可妮/杨笠 | 优酷 YOUKU
   ✨ Moved to TV: Shanghai Share Life - S01E01.mkv

✅ All Tasks Finished (0m 52s)
